<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/model-template/MachineHack_longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 17.4MB/s 
     |████████████████████████████████| 3.0MB 26.1MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9bf93c10e1454a81b38d7732518895414674642ac3f3c58ce6d50104a03e207b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-15 13:23:41--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.66.60
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.66.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  12.7MB/s    in 9.3s    

2020-10-15 13:23:51 (10.5 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_extra

In [3]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [4]:
#import pandas as pd
import numpy as np
import pandas as pd

In [5]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")

In [6]:
test_label_df = pd.read_csv('/content/ensemble_v8_1.csv')
test_df['label'] = test_label_df['label']

In [7]:
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

In [8]:
train_data.shape

(450000, 4)

In [9]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [10]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "allenai/longformer-base-4096"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.127.109.122:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.127.109.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.127.109.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [11]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = AlbertTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
#tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = ElectraTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [12]:
def build_model():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL)
  #bert_model = TFAutoModel.from_pretrained('/content/github-albert-xlarge-v2/')
  #bert_model = TFAutoModel.from_pretrained('../../github-roberta-large/')
  output_layer = bert_model(input_layer) 
  #pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[0])
  output = tf.keras.layers.Dense(3, activation='softmax')(output_layer[1])
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()
    
  return classifier_model

In [13]:
all_train = train_data[0:200000]
all_train = all_train.append(train_data[230000:])

In [14]:
question_df = all_train[all_train.label==2]
bug_issue = all_train[all_train.label!=2]
bug_issue = bug_issue.sample(100000)
train = question_df.append(bug_issue)
train = train.append(test_df)
train = train.sample(len(train))

In [15]:
train.label.value_counts()

1    65048
0    62615
2    41662
Name: label, dtype: int64

In [16]:
validate = train_data[200000:230000]
validate.label.value_counts()

1    13846
0    13278
2     2876
Name: label, dtype: int64

In [17]:
with strategy.scope():
    classifier_model = build_model()

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
tf_longformer_model (TFLongf ((None, 320, 768), (None, 148659456 
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 148,661,763
Trainable params: 148,661,763
Non-trainable params: 0
_________________________________________________________________


In [18]:
# data preparation
X_data = regular_encode(train.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_train = train.label.values
print(X_data.shape, y_train.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(169325, 320) (169325,)


In [19]:
# data preparation
X_val = regular_encode(validate.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_val = validate.label.values
print(X_val.shape, y_val.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(30000, 320) (30000,)


In [20]:
X_test = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [21]:
# 100K datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


InvalidArgumentError: ignored

In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/3427 [..............................] - ETA: 10:52 - loss: 0.4741 - accuracy: 0.8359WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0135s vs `on_train_batch_end` time: 0.1806s). Check your callbacks.


3427/3427 [==============================] - ETA: 0s - loss: 0.4342 - accuracy: 0.8384WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0060s vs `on_test_batch_end` time: 0.0515s). Check your callbacks.


3427/3427 [==============================] - 708s 207ms/step - loss: 0.4342 - accuracy: 0.8384 - val_loss: 0.4419 - val_accuracy: 0.8326


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/2646 [..............................] - ETA: 8:26 - loss: 0.3812 - accuracy: 0.8672WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0135s vs `on_train_batch_end` time: 0.1827s). Check your callbacks.


2646/2646 [==============================] - ETA: 0s - loss: 0.4074 - accuracy: 0.8488WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0047s vs `on_test_batch_end` time: 0.0507s). Check your callbacks.


2646/2646 [==============================] - 586s 222ms/step - loss: 0.4074 - accuracy: 0.8488 - val_loss: 0.4582 - val_accuracy: 0.8312


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/1865 [..............................] - ETA: 5:59 - loss: 0.3776 - accuracy: 0.8594WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0157s vs `on_train_batch_end` time: 0.1858s). Check your callbacks.


1865/1865 [==============================] - ETA: 0s - loss: 0.3585 - accuracy: 0.8690WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0055s vs `on_test_batch_end` time: 0.0511s). Check your callbacks.


1865/1865 [==============================] - 431s 231ms/step - loss: 0.3585 - accuracy: 0.8690 - val_loss: 0.5275 - val_accuracy: 0.8155


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/1552 [..............................] - ETA: 4:53 - loss: 0.3028 - accuracy: 0.9297WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0138s vs `on_train_batch_end` time: 0.1791s). Check your callbacks.


1552/1552 [==============================] - ETA: 0s - loss: 0.2983 - accuracy: 0.8939WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0048s vs `on_test_batch_end` time: 0.0524s). Check your callbacks.


1552/1552 [==============================] - 336s 216ms/step - loss: 0.2983 - accuracy: 0.8939 - val_loss: 0.6123 - val_accuracy: 0.7819


In [ ]:
y_val_pred = classifier_model.predict(X_val, batch_size=global_batch_size, verbose=1)
y_test_pred = classifier_model.predict(X_test, batch_size=global_batch_size, verbose=1)

  2/235 [..............................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0068s vs `on_predict_batch_end` time: 0.0849s). Check your callbacks.


  2/235 [..............................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_predict_batch_end` time: 0.0836s). Check your callbacks.


235/235 [==============================] - 21s 88ms/step


In [ ]:
eval_df = pd.DataFrame({'X':y_val_pred.tolist(), 'y':y_val.tolist()})

In [ ]:
eval_df.to_csv('bert-base-cased-maxlen320-PL-6epochs-90K87K40K-eval-prob.csv', index=False)

In [ ]:
submission = pd.read_csv('sample submission.csv')
submission['label'] = y_test_pred.tolist()
file_name = 'bert-base-cased-maxlen320-PL-6epochs-90K87K40K-test-prob.csv'
submission.to_csv(file_name, index=False)